#### CAFA6-08-Model Improvement

Improvement from CAFA6-07: 20 features to 320 features

Future direction: Use model ensemble

References:
- https://www.kaggle.com/code/analyticaobscura/cafa-6-decoding-protein-mysteries
- (ESM-2 embeddings 320 features) https://www.kaggle.com/code/dalloliogm/compute-protein-embeddings-with-esm2-esm-c/notebook
- (MLP with ESM2) https://www.kaggle.com/code/jwang2025learning/cafa-6-function-prediction-using-prott5?scriptVersionId=282801093

---

In [54]:
!pip install biopython > /dev/null

## Step 1: Load CAFA6 files

---

In [55]:
from Bio import SeqIO  # parse fasta file
import pandas as pd
import numpy as np

In [56]:
# CAFA6 file paths
TRAIN_TERMS = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv"
TRAIN_SEQ = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
TRAIN_TAXONOMY = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv"
TEST_SEQ = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta"

In [57]:
# Dict {entryId, seq}
train_sequences = {rec.id: str(rec.seq) for rec in SeqIO.parse(TRAIN_SEQ, 'fasta')}
test_sequences  = {rec.id: str(rec.seq) for rec in SeqIO.parse(TEST_SEQ,  'fasta')}

print(f'Loaded {len(train_sequences)} train and {len(test_sequences)} test sequences')

Loaded 82404 train and 224309 test sequences


In [58]:
print("Train dict:", list(train_sequences.items())[0])
print("Test dict:", list(test_sequences.items())[0])

Train dict: ('sp|A0A0C5B5G6|MOTSC_HUMAN', 'MRWQEMGYIFYPRKLR')
Test dict: ('A0A0C5B5G6', 'MRWQEMGYIFYPRKLR')


In [59]:
train_ids = [i.split('|')[1] for i in train_sequences.keys()]
test_ids = list(test_sequences.keys())

## Step 2: Feature extraction
  
---

In [60]:
# Embeddings file paths
ESM_EMBEDDINGS = "/kaggle/input/esm2-t6-8m-ur50d-cafa6"
TRAIN_EMBEDDINGS = ESM_EMBEDDINGS + "/protein_embeddings_train.npy"
TEST_EMBEDDINGS = ESM_EMBEDDINGS + "/protein_embeddings_test.npy"

In [61]:
# Load embeddings
X_train = np.load(TRAIN_EMBEDDINGS)
X_test = np.load(TEST_EMBEDDINGS)

In [62]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (82404, 320)
X_test shape: (224309, 320)


In [63]:
# Normalization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
print("X_train_norm shape:", X_train_norm.shape)
print("X_test_norm shape:", X_test_norm.shape)

X_train_norm shape: (82404, 320)
X_test_norm shape: (224309, 320)


## Step 3: Customized MLP

---

In [64]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [65]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.3),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.25),

            nn.Linear(1024, output_dim)  # logits
        )

    def forward(self, x):
        return self.net(x)

## Step 4: Label encoding

We divide all labels into three subsets (one for MF, one for BP, and one for CC).
-  As a result, for each sequence, we will have 3 vectors as follows using multi-hot encoding (i.e. simply one-hot for multi-classification problems)
    - For MF: [0 1 0 1 ... 0] of length num_unique(MF_GO_temrs)
    - For BP: [0 1 0 1 ... 0] of length num_unique(BP_GO_temrs)
    - For CC: [0 1 0 1 ... 0] of length num_unique(CC_GO_temrs)
- Then we will train three separate models for each ontology. We use three models to predict for a single example in the test set, and gather the predictions.

---

In [66]:
# Create three label sets
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

y_trains = dict()
mlb_dict = dict()
models = dict()

train_terms_df = pd.read_csv(TRAIN_TERMS, sep="\t")

for aspect in ['P', 'C', 'F']:
    # Filter the train_terms_df based on aspect
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect]

    # Group the dataFrame based on the EntryID, turn all the GO terms to a list, finally turns this dataFrame to a dict {entryID: terms}
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()

    # Create a list of labels for this aspect, if an entryID doesn't exist in this aspect, give it a []
    # This ensures y_train is of shape (82404, ...)
    labels = [protein_terms.get(entry_id, []) for entry_id in train_ids]

    # Multi-hot encoding, use sparse representation
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train = mlb.fit_transform(labels)
    y_trains[aspect] = y_train
    
    print(f"y_train shape for {aspect} ontology: {y_train.shape} \t\t Number of unique {aspect} terms: {y_train.shape[1]}")

    # Save to dict
    mlb_dict[aspect] = mlb
    model = MLPClassifier(input_dim=X_train.shape[1], output_dim=y_train.shape[1])
    models[aspect] = model

y_train shape for P ontology: (82404, 16858) 		 Number of unique P terms: 16858
y_train shape for C ontology: (82404, 2651) 		 Number of unique C terms: 2651
y_train shape for F ontology: (82404, 6616) 		 Number of unique F terms: 6616


In [67]:
models

{'P': MLPClassifier(
   (net): Sequential(
     (0): Linear(in_features=320, out_features=2048, bias=True)
     (1): ReLU()
     (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (3): Dropout(p=0.3, inplace=False)
     (4): Linear(in_features=2048, out_features=1024, bias=True)
     (5): ReLU()
     (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (7): Dropout(p=0.25, inplace=False)
     (8): Linear(in_features=1024, out_features=16858, bias=True)
   )
 ),
 'C': MLPClassifier(
   (net): Sequential(
     (0): Linear(in_features=320, out_features=2048, bias=True)
     (1): ReLU()
     (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (3): Dropout(p=0.3, inplace=False)
     (4): Linear(in_features=2048, out_features=1024, bias=True)
     (5): ReLU()
     (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (7): Dropout(p=0.25

In [68]:
y_trains

{'P': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 250805 stored elements and shape (82404, 16858)>,
 'C': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 157770 stored elements and shape (82404, 2651)>,
 'F': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 128452 stored elements and shape (82404, 6616)>}

## Step 5: Model training

In [69]:
# DataLoader
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32)

loaders = {}

for aspect in ['P', 'C', 'F']:
    # Convert sparse CSR → dense numpy → float tensor
    y_dense = y_trains[aspect].toarray().astype('float32')
    y_tensor = torch.tensor(y_dense, dtype=torch.float32)

    dataset = TensorDataset(X_train_tensor, y_tensor)

    loaders[aspect] = DataLoader(dataset, batch_size=128, shuffle=True)

    print(aspect, "loader ready:", y_tensor.shape)

P loader ready: torch.Size([82404, 16858])
C loader ready: torch.Size([82404, 2651])
F loader ready: torch.Size([82404, 6616])


In [70]:
# Training
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()

def train_one_model(model, loader, epochs=5, lr=1e-3, device='cuda'):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for ep in range(epochs):
        total_loss = 0.0

        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            logits = model(X_batch)               # shape: (batch, num_labels)
            loss = criterion(logits, y_batch)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {ep+1}/{epochs}   Loss = {total_loss/len(loader):.4f}")

    return model

In [71]:
# Train 3 models
trained_models = dict()

for aspect in ['P', 'C', 'F']:
    print("\nTraining", aspect, "...")
    trained_models[aspect] = train_one_model(
        model=models[aspect],
        loader=loaders[aspect],
        epochs=8,
        lr=1e-3,
        device=device
    )


Training P ...
Epoch 1/8   Loss = 0.0661
Epoch 2/8   Loss = 0.0017
Epoch 3/8   Loss = 0.0015
Epoch 4/8   Loss = 0.0015
Epoch 5/8   Loss = 0.0014
Epoch 6/8   Loss = 0.0014
Epoch 7/8   Loss = 0.0014
Epoch 8/8   Loss = 0.0013

Training C ...
Epoch 1/8   Loss = 0.0679
Epoch 2/8   Loss = 0.0038
Epoch 3/8   Loss = 0.0036
Epoch 4/8   Loss = 0.0034
Epoch 5/8   Loss = 0.0033
Epoch 6/8   Loss = 0.0032
Epoch 7/8   Loss = 0.0032
Epoch 8/8   Loss = 0.0031

Training F ...
Epoch 1/8   Loss = 0.0658
Epoch 2/8   Loss = 0.0016
Epoch 3/8   Loss = 0.0014
Epoch 4/8   Loss = 0.0013
Epoch 5/8   Loss = 0.0012
Epoch 6/8   Loss = 0.0012
Epoch 7/8   Loss = 0.0011
Epoch 8/8   Loss = 0.0011


## Step 6: Inference and Submission

In [72]:
BATCH_SIZE = 5000  # avoid memory overflow
submission_list = []

for i in tqdm(range(0, len(test_ids), BATCH_SIZE), desc="Predicting on Test Set"):
    batch_entry_ids = test_ids[i : i + BATCH_SIZE]

    # Slice features
    X_batch = X_test_norm[i : i + BATCH_SIZE]
    X_batch = torch.tensor(X_batch, dtype=torch.float32, device=device)

    # For each ontology aspect (P, F, C)
    for aspect, model in trained_models.items():
        model.eval()

        # Forward pass (logits → probabilities)
        with torch.no_grad():
            logits = model(X_batch)
            probs = torch.sigmoid(logits).cpu().numpy()  # ndarray (batch, num_labels)

        mlb = mlb_dict[aspect]  # MultiLabelBinarizer for this aspect

        # Loop over proteins in the batch
        for j, entry_id in enumerate(batch_entry_ids):
            prob_vec = probs[j]

            # threshold at 0.02
            candidate_indices = np.where(prob_vec > 0.02)[0]

            for idx in candidate_indices:
                submission_list.append(
                    (entry_id, mlb.classes_[idx], round(prob_vec[idx], 3))
                )

Predicting on Test Set: 100%|██████████| 45/45 [01:10<00:00,  1.57s/it]


In [73]:
submission_df = pd.DataFrame(
    submission_list,
    columns=['Protein Id', 'GO Term Id', 'Prediction']
)

submission_df = submission_df.sort_values(
    by=['Protein Id', 'Prediction'],
    ascending=[True, False]
)

# Limit 1500 predictions per protein
final_submission_df = (
    submission_df.groupby('Protein Id')
    .head(1500)
    .reset_index(drop=True)
)

final_submission_df.to_csv('submission.tsv', sep='\t', index=False, header=False)

In [74]:
print("\nSubmission file 'submission.tsv' created successfully.")
print(f"Total predictions in final submission: {len(final_submission_df):,}")
print("Submission DataFrame Head:")
display(final_submission_df.head())


Submission file 'submission.tsv' created successfully.
Total predictions in final submission: 5,784,652
Submission DataFrame Head:


,Protein Id,GO Term Id,Prediction
0,A0A017SE81,GO:0005515,0.158
1,A0A017SE81,GO:0005886,0.101
2,A0A017SE81,GO:0004745,0.084
3,A0A017SE81,GO:0071768,0.069
4,A0A017SE81,GO:0016020,0.053
